Set of script to do web crawling in a batch for website https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line

In [2]:
import requests
from pyquery import PyQuery as pq
from fake_useragent import UserAgent
import random
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from datetime import date, timedelta
import pandas as pd
import time

In [3]:
def get_web_content(url , data = None):
    
    ua = UserAgent()
    header={'user-agent': ua.random}
    timeout = random.choice(range(80, 180))
    while True:
        try:
            rep = requests.get(url,headers = header,timeout = timeout)
            rep.encoding = 'utf-8'
            # req = urllib.request.Request(url, data, header)
            # response = urllib.request.urlopen(req, timeout=timeout)
            # html1 = response.read().decode('UTF-8', errors='ignore')
            # response.close()
            break
        # except urllib.request.HTTPError as e:
        #         print( '1:', e)
        #         time.sleep(random.choice(range(5, 10)))
        #
        # except urllib.request.URLError as e:
        #     print( '2:', e)
        #     time.sleep(random.choice(range(5, 10)))
        except socket.timeout as e:
            print( '3:', e)
            time.sleep(random.choice(range(8,15)))

        except socket.error as e:
            print( '4:', e)
            time.sleep(random.choice(range(20, 60)))

        except http.client.BadStatusLine as e:
            print( '5:', e)
            time.sleep(random.choice(range(30, 80)))

        except http.client.Incomplete|Read as e:
            print( '6:', e)
            time.sleep(random.choice(range(5, 15)))

    return rep.text
    # return html_text

In [4]:
def get_team_data(html_text):
    final = []
    bs = BeautifulSoup(html_text, "html.parser")  # 创建BeautifulSoup对象
    body = bs.body # 获取body部分
    team_data = body.find_all('span', {'class':'team-name'}) 
    if team_data == None:
        return []

    for i in team_data: 
        final.append(i.a.string)
    return final

def get_odds_data(html_text):
    final = []
    bs = BeautifulSoup(html_text, "html.parser")  # 创建BeautifulSoup对象
    body = bs.body # 获取body部分
    odds_data = body.find_all('div', {'class':'el-div eventLine-book'})
    if odds_data == None:
        return []
    number = 0
    for i in odds_data: 
        odds_data2 = i.find_all('div', {'class':'eventLine-book-value'}) 

        for j in odds_data2: 
            if number%8 == 0 or number%8 == 1:
                final.append(int(j.string))
            number+= 1
    return final
def get_date(html_text):
    MONTH_MAPPING = {
    "January": "01",
    "February": "02",
    "March": "03",
    "April": "04",
    "May": "05",
    "June": "06",
    "July": "07",
    "August": "08",
    "September": "09",
    "October": "10",
    "November": "11",
    "December": "12"}
    final = []
    bs = BeautifulSoup(html_text, "html.parser")  # 创建BeautifulSoup对象
    body = bs.body # 获取body部分
    data = body.find('div', {'class':'date'})
    if data == None:
        return None
    #inal.append(data.find("div"))
    date = data.string
    temp = date.split(",")
    year = temp[2].replace(" ", "")
    month = MONTH_MAPPING[temp[1][1:].split(" ")[0]]
    day = temp[1][1:].split(" ")[1]
    if len(day) == 1:
        day = "".join(["0", day])
    date_num_string = "".join([year, month, day])
    return date_num_string

In [5]:
nba_odds_url_base = "https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date="

start_date = date(2019, 8, 20) 
end_date = date(2022, 3, 26)    # perhaps date.now()

delta = end_date - start_date   # returns timedelta

date_string_list = []
date_list = []
for i in range(delta.days + 1):
    day = start_date + timedelta(days=i)
    date_list.append(day)
    date_string_list.append(day.strftime("%Y%m%d"))

In [6]:
nba_odds_url_list = []
for date_string in date_string_list:
    nba_odds_url = "".join([nba_odds_url_base, date_string])
    nba_odds_url_list.append(nba_odds_url)

In [7]:
nba_odds_url_list[0:5]

['https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20190820',
 'https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20190821',
 'https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20190822',
 'https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20190823',
 'https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20190824']

In [8]:
date_string_list[0:5]

['20190820', '20190821', '20190822', '20190823', '20190824']

In [9]:
date_list[0:5]

[datetime.date(2019, 8, 20),
 datetime.date(2019, 8, 21),
 datetime.date(2019, 8, 22),
 datetime.date(2019, 8, 23),
 datetime.date(2019, 8, 24)]

In [10]:
import logging
# create logger with name
# if not specified, it will be root
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)

# create a handler, write to log.txt
# logging.FileHandler(self, filename, mode='a', encoding=None, delay=0)
# A handler class which writes formatted logging records to disk files.
fh = logging.FileHandler('log.txt')
fh.setLevel(logging.DEBUG)

# create another handler, for stdout in terminal
# A handler class which writes logging records to a stream
sh = logging.StreamHandler()
sh.setLevel(logging.DEBUG)

# set formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
sh.setFormatter(formatter)

# add handler to logger
logger.addHandler(fh)
logger.addHandler(sh)


odds_data_dict = {"date": [], "team": [], "odds": []}
for index, url in enumerate(nba_odds_url_list):
    logger.info("Getting data from url: \n {} \n".format(url))
    try:
        r = get_web_content(url)
        if r is None:
            logger.info("web request returns None. {} does not have odds data".format(date))
            continue
        
        date = get_date(r)
        logger.info("date from website is {}".format(date))
        if date != date_string_list[index]:
            logger.info("{} does not have nba games".format(date_string_list[index]))
            continue
        
        team_data = get_team_data(r)
        odds_data = get_odds_data(r)
        if len(team_data) != len(odds_data):
            logger.error("team number does not match odds number")
            continue
        
        odds_data_dict["date"].extend([date_list[index].strftime("%Y-%m-%d")]*len(team_data))
        odds_data_dict["team"].extend(team_data)
        odds_data_dict["odds"].extend(odds_data)
        logger.info("successful")
    except Exception as e:
        logger.exception(e)
        continue
     

2022-03-26 21:20:29,085 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20190820 

2022-03-26 21:20:29,701 - my_logger - INFO - date from website is None
2022-03-26 21:20:29,702 - my_logger - INFO - 20190820 does not have nba games
2022-03-26 21:20:29,703 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20190821 

2022-03-26 21:20:30,331 - my_logger - INFO - date from website is None
2022-03-26 21:20:30,332 - my_logger - INFO - 20190821 does not have nba games
2022-03-26 21:20:30,333 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20190822 

2022-03-26 21:20:30,902 - my_logger - INFO - date from website is None
2022-03-26 21:20:30,902 - my_logger - INFO - 20190822 does not have nba games
2022-03-26 21:20:30,903 - my_logger - INFO - Getting data from u

2022-03-26 21:20:46,415 - my_logger - INFO - date from website is None
2022-03-26 21:20:46,416 - my_logger - INFO - 20190915 does not have nba games
2022-03-26 21:20:46,417 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20190916 

2022-03-26 21:20:47,064 - my_logger - INFO - date from website is None
2022-03-26 21:20:47,065 - my_logger - INFO - 20190916 does not have nba games
2022-03-26 21:20:47,066 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20190917 

2022-03-26 21:20:47,712 - my_logger - INFO - date from website is None
2022-03-26 21:20:47,713 - my_logger - INFO - 20190917 does not have nba games
2022-03-26 21:20:47,714 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20190918 

2022-03-26 21:20:48,327 - my_logger - INFO - date from website i

2022-03-26 21:20:59,936 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20191007 

2022-03-26 21:21:00,595 - my_logger - INFO - date from website is 20191007
2022-03-26 21:21:00,710 - my_logger - ERROR - int() argument must be a string, a bytes-like object or a number, not 'NoneType'
Traceback (most recent call last):
  File "<ipython-input-10-727627e692aa>", line 44, in <module>
    odds_data = get_odds_data(r)
  File "<ipython-input-4-110c63b99cff>", line 26, in get_odds_data
    final.append(int(j.string))
TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'
2022-03-26 21:21:00,711 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20191008 

2022-03-26 21:21:01,460 - my_logger - INFO - date from website is 20191008
2022-03-26 21:21:01,567 - my_logger - ERROR - int() argument must be a stri

2022-03-26 21:21:09,873 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20191019 

2022-03-26 21:21:10,464 - my_logger - INFO - date from website is None
2022-03-26 21:21:10,465 - my_logger - INFO - 20191019 does not have nba games
2022-03-26 21:21:10,466 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20191020 

2022-03-26 21:21:11,091 - my_logger - INFO - date from website is None
2022-03-26 21:21:11,092 - my_logger - INFO - 20191020 does not have nba games
2022-03-26 21:21:11,093 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20191021 

2022-03-26 21:21:11,777 - my_logger - INFO - date from website is None
2022-03-26 21:21:11,778 - my_logger - INFO - 20191021 does not have nba games
2022-03-26 21:21:11,780 - my_logger - INFO - Getting data from u

2022-03-26 21:21:21,608 - my_logger - INFO - date from website is 20191102
2022-03-26 21:21:21,710 - my_logger - ERROR - int() argument must be a string, a bytes-like object or a number, not 'NoneType'
Traceback (most recent call last):
  File "<ipython-input-10-727627e692aa>", line 44, in <module>
    odds_data = get_odds_data(r)
  File "<ipython-input-4-110c63b99cff>", line 26, in get_odds_data
    final.append(int(j.string))
TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'
2022-03-26 21:21:21,711 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20191103 

2022-03-26 21:21:22,429 - my_logger - INFO - date from website is 20191103
2022-03-26 21:21:22,514 - my_logger - ERROR - int() argument must be a string, a bytes-like object or a number, not 'NoneType'
Traceback (most recent call last):
  File "<ipython-input-10-727627e692aa>", line 44, in <module>
    odds_data

...

2022-03-26 21:31:01,494 - my_logger - INFO - date from website is 20211120
2022-03-26 21:31:01,603 - my_logger - INFO - successful
2022-03-26 21:31:01,604 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20211121 

2022-03-26 21:31:02,249 - my_logger - INFO - date from website is 20211121
2022-03-26 21:31:02,339 - my_logger - INFO - successful
2022-03-26 21:31:02,341 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20211122 

2022-03-26 21:31:03,046 - my_logger - INFO - date from website is 20211122
2022-03-26 21:31:03,159 - my_logger - ERROR - int() argument must be a string, a bytes-like object or a number, not 'NoneType'
Traceback (most recent call last):
  File "<ipython-input-10-727627e692aa>", line 44, in <module>
    odds_data = get_odds_data(r)
  File "<ipython-input-4-110c63b99cff>", line 26, in get_odds_data
    final.ap

2022-03-26 21:31:24,276 - my_logger - INFO - date from website is 20211217
2022-03-26 21:31:24,400 - my_logger - INFO - successful
2022-03-26 21:31:24,402 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20211218 

2022-03-26 21:31:25,134 - my_logger - INFO - date from website is 20211218
2022-03-26 21:31:25,231 - my_logger - INFO - successful
2022-03-26 21:31:25,233 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20211219 

2022-03-26 21:31:25,975 - my_logger - INFO - date from website is 20211219
2022-03-26 21:31:26,097 - my_logger - INFO - successful
2022-03-26 21:31:26,098 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20211220 

2022-03-26 21:31:26,855 - my_logger - INFO - date from website is 20211220
2022-03-26 21:31:26,947 - my_logger - INFO 

2022-03-26 21:31:48,707 - my_logger - INFO - date from website is 20220114
2022-03-26 21:31:48,821 - my_logger - INFO - successful
2022-03-26 21:31:48,822 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20220115 

2022-03-26 21:31:49,565 - my_logger - INFO - date from website is 20220115
2022-03-26 21:31:49,720 - my_logger - ERROR - int() argument must be a string, a bytes-like object or a number, not 'NoneType'
Traceback (most recent call last):
  File "<ipython-input-10-727627e692aa>", line 44, in <module>
    odds_data = get_odds_data(r)
  File "<ipython-input-4-110c63b99cff>", line 26, in get_odds_data
    final.append(int(j.string))
TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'
2022-03-26 21:31:49,722 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20220116 

2022-03-26 21:31:50

2022-03-26 21:32:09,042 - my_logger - INFO - date from website is 20220206
2022-03-26 21:32:09,142 - my_logger - INFO - successful
2022-03-26 21:32:09,143 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20220207 

2022-03-26 21:32:09,836 - my_logger - INFO - date from website is 20220207
2022-03-26 21:32:09,920 - my_logger - INFO - successful
2022-03-26 21:32:09,920 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20220208 

2022-03-26 21:32:10,738 - my_logger - INFO - date from website is 20220208
2022-03-26 21:32:10,858 - my_logger - INFO - successful
2022-03-26 21:32:10,858 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20220209 

2022-03-26 21:32:11,560 - my_logger - INFO - date from website is 20220209
2022-03-26 21:32:11,654 - my_logger - INFO 

2022-03-26 21:32:29,916 - my_logger - INFO - successful
2022-03-26 21:32:29,917 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20220305 

2022-03-26 21:32:30,583 - my_logger - INFO - date from website is 20220305
2022-03-26 21:32:30,676 - my_logger - INFO - successful
2022-03-26 21:32:30,677 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20220306 

2022-03-26 21:32:31,457 - my_logger - INFO - date from website is 20220306
2022-03-26 21:32:31,563 - my_logger - INFO - successful
2022-03-26 21:32:31,564 - my_logger - INFO - Getting data from url: 
 https://classic.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date=20220307 

2022-03-26 21:32:32,347 - my_logger - INFO - date from website is 20220307
2022-03-26 21:32:32,451 - my_logger - INFO - successful
2022-03-26 21:32:32,451 - my_logger - INFO - Getting data from

In [12]:
# clean up file handlers
handlers = logger.handlers[:]
for handler in handlers:
    handler.close()
    logger.removeHandler(handler)

In [11]:
pd.DataFrame(odds_data_dict).to_csv("odds_data.csv", index=False)